In [63]:
"""
Script written by Andrew Rohne. This prepares updates the joint tour model for Met Council

Needs: 
  1: Visualizer Survey folder - reads one of the CSVs
  2: ActivitySim tours output

How To Use:
  1: Set the project_dir (cell 2) to the base of your project - this should include the visualizer summary inputs and the model data
  2: Set the output folder (where some CSV files are saved)
  2: Fix the census_auto_ownership_file (text should be relative to project_dir)
  3: Fix the hh file (again, should be relative to project_dir)
  4: You'll probably need to update some fields
  

"""
import sys
import pandas as pd
import os
import numpy as np
pd.options.display.max_columns = 500
pd.options.display.max_rows = 500

SAMPLE_RATE = 100000.0 / 1527646.0

MODEL_OUTPUT = r'E:\Met_Council\metc-asim-model\Base_2018\activitysim\output'
MODEL_INPUT = r"E:\Met_Council\metc-asim-model\input"

vis_survey_summaries = r'E:\Met_Council\metc-asim-model\Input\Visualizer\base'
asim_config_folder = r'E:\Met_Council\metc-asim-model\source\activitysim\configs'
lu = pd.read_csv(os.path.join(MODEL_INPUT, "socioeconomic", "land_use.csv"))

PERTYPEMAP = {1: 'FT', 2: 'PT', 3: 'U', 4: 'NW', 5: 'R', 6: 'SD', 7: 'SPD', 8: 'PS'}
coef_name_map = {'1_day_week': 'coef_Calib_1dpw_pc', '2_3_days_week': 'coef_Calib_23dpw_pc', '4_days_week': 'coef_Calib_4pdpw_pc', 'No_Telecommute': 'coef_Calib_no'}

modTC_in = pd.read_csv(os.path.join(MODEL_OUTPUT, "final_persons.csv"))
obsTCF = pd.read_csv(os.path.join(vis_survey_summaries, "telecommuteFrequency.csv")).rename(columns = {'freq': 'ObsTC'})
asim_config = pd.read_csv(os.path.join(asim_config_folder, "telecommute_frequency_coefficients.csv"))



In [64]:
modTC = modTC_in[~modTC_in['telecommute_frequency'].isna()].groupby('telecommute_frequency').agg(ModW = ('person_id', 'count')).reset_index()
modTC['ModW'] = modTC['ModW'] / SAMPLE_RATE
modTC['modPct'] = modTC['ModW'] / modTC['ModW'].sum()
obsTCF['obsPct'] = obsTCF['ObsTC'] / obsTCF['ObsTC'].sum()
TCCompare = modTC.merge(obsTCF, on = 'telecommute_frequency')

TCCompare['coefAdj'] = np.log(TCCompare['obsPct'] / TCCompare['modPct'])
TCCompare['coefficient_name'] = TCCompare['telecommute_frequency'].map(coef_name_map)
TCCompare = TCCompare.merge(asim_config.rename(columns = {'value': 'old_coefficient', 'coefficient_name.1': 'description'}), how = 'left', on = 'coefficient_name')

TCCompare[['telecommute_frequency', 'ObsTC', 'ModW', 'obsPct', 'modPct']].style.format(formatter = {'ObsTC': "{:,.0f}", 'ModW': "{:,.0f}", 'obsPct': "{:.2%}".format, 'modPct': "{:.2%}".format})

,telecommute_frequency,ObsTC,ModW,obsPct,modPct
0,1_day_week,"62,672","231,683",4.34%,4.26%
1,2_3_days_week,"155,111","560,035",10.74%,10.31%
2,4_days_week,"221,471","800,624",15.34%,14.74%
3,No_Telecommute,"1,004,545","3,840,624",69.58%,70.69%


In [65]:
TCCompare['value'] = TCCompare['old_coefficient'] + TCCompare['coefAdj']
TCCompare.loc[TCCompare['constrain'] == 'T', 'value'] = TCCompare['old_coefficient']
TCCompare

,telecommute_frequency,ModW,modPct,Unnamed: 0,ObsTC,obsPct,coefAdj,coefficient_name,description,old_coefficient,constrain,value
0,1_day_week,2.316828e+05,0.042644,1,6.267165e+04,0.043407,0.017748,coef_Calib_1dpw_pc,Calib TC 1 day per week - post-Covid,-0.577885,F,-0.560138
1,2_3_days_week,5.600350e+05,0.103081,2,1.551108e+05,0.107432,0.041348,coef_Calib_23dpw_pc,Calib TC 2-3 days per week - post-Covid,0.729249,F,0.770597
2,4_days_week,8.006240e+05,0.147364,3,2.214707e+05,0.153394,0.040107,coef_Calib_4pdpw_pc,Calib TC 4+ days per week - post-Covid,1.407752,F,1.447858
3,No_Telecommute,3.840624e+06,0.706911,4,1.004545e+06,0.695766,-0.015892,coef_Calib_no,Calib - no TC,-0.100109,T,-0.100109


In [66]:
TCCompare[['coefficient_name', 'description', 'value', 'constrain']].to_csv(sys.stdout, index = False, header = False)

coef_Calib_1dpw_pc,Calib TC 1 day per week - post-Covid,-0.560137575141382,F
coef_Calib_23dpw_pc,Calib TC 2-3 days per week - post-Covid,0.7705965174365309,F
coef_Calib_4pdpw_pc,Calib TC 4+ days per week - post-Covid,1.447858491918306,F
coef_Calib_no,Calib - no TC,-0.1001087229466148,T
